<H3>
    
**(Implementation project) Using a programming language that you are familiar with, such as C++ or Java, implement three frequent itemset mining algorithms introduced in this chapter: (1) Apriori [AS94b], (2) FP-growth [HPY00],** 
 
 </H3>

Apriori Function

This is the main function of this Apriori Python implementation. The most important part of this function is from line 16 ~ line 21. It basically follows my modified pseudocode written above.

1. Generate the candidate set by joining the frequent itemset from the previous stage.

2. Perform subset testing and prune the candidate set if there’s an infrequent itemset contained.

3. Calculate the final frequent itemset by getting those satisfy minimum support.

In [146]:
from collections import defaultdict
from itertools import chain, combinations

def apriori(itemSetList, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    # Final result, global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    #rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    #rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules


Candidate Generation

For self-joining, we simply get all the union through brute-force and only return those are in the specific length.


In [147]:
def getUnion(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

Pruning

To perform subset testing, we loop through all possible subsets in the itemset. If the subset is not in the previous frequent itemset, we prune it.


In [148]:
def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet

Get Frequent Itemset from Candidate

In the final step, we turn the candidate sets into frequent itemsets. Since we are not applying any improvement technique. The only approach we can go for is to brainlessly loop through the item and itemset over and over again to obtain the count. At last, we only retain the itemsets whose support is equal or higher than minimum support.

In [149]:
def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList))
        if(support >= minSup):
            freqItemSet.add(item)

    return freqItemSet

In [150]:
# To get the itemset from the list
def getItemSetFromList(itemSetList):
    tempItemSet = set()

    for itemSet in itemSetList:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))

    return tempItemSet

In [151]:
def associationRule(freqItemSet, itemSetWithSup, minConf):
    rules = []
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            for s in subsets:
                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules

In [152]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

In [142]:
itemSetList = [['eggs', 'bacon', 'soup'],
                ['eggs', 'bacon', 'apple'],
                ['soup', 'bacon', 'banana']]
freqItemSet, rules = apriori(itemSetList, minSup=0.5, minConf=0.5)
print(freqItemSet)
print(rules) 

{1: {frozenset({'soup'}), frozenset({'eggs'}), frozenset({'bacon'})}, 2: {frozenset({'bacon', 'eggs'}), frozenset({'bacon', 'soup'})}}
[]


In [143]:
itemSetList1 = [['M', 'O', 'N','K','E','Y'],
                ['D', 'O', 'N','K','E','Y'],
                ['M', 'O', 'N','K','E','Y'],
                ['M', 'A','K','E'],
                ['M', 'U','C','K','Y'],
                ['C', 'O','O','K','I','E']
              ]
freqItemSet1, rules1 = apriori(itemSetList1, minSup=0.6, minConf=0.8)
print(freqItemSet1)


{1: {frozenset({'O'}), frozenset({'K'}), frozenset({'Y'}), frozenset({'M'}), frozenset({'E'})}, 2: {frozenset({'O', 'E'}), frozenset({'K', 'O'}), frozenset({'K', 'Y'}), frozenset({'K', 'E'}), frozenset({'K', 'M'})}, 3: {frozenset({'K', 'O', 'E'})}}


In [144]:
rules1

[]

In [42]:
#rules1

[[{'K'}, {'E'}, 0.8333333333333334],
 [{'O'}, {'E'}, 1.0],
 [{'O'}, {'K'}, 1.0],
 [{'Y'}, {'K'}, 1.0],
 [{'E'}, {'K'}, 1.0],
 [{'M'}, {'K'}, 1.0],
 [{'O'}, {'E', 'K'}, 1.0],
 [{'K', 'O'}, {'E'}, 1.0],
 [{'E', 'O'}, {'K'}, 1.0]]

In [153]:
import pandas as pd
colnames=['age','workclass','fnlwgt',
'education', 'education-num','marital-status', 'occupation', 'relationship','race','sex','capital-gain','capital-loss','hours-per-week',
'native-country','Salary'] 

itemSetList1 =pd.read_csv("testr.csv", names=colnames, header=None)

#freqItemSet1, rules1 = apriori(itemSetList1, minSup=0.6, minConf=0.8)
#print(freqItemSet1)


In [154]:
itemSetList1

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [111]:
'''observations = [] 
for i in range(len(itemSetList1)):
    observations.append([str(itemSetList1.values[i,j]) for j in range(15)])'''

'observations = [] \nfor i in range(len(itemSetList1)):\n    observations.append([str(itemSetList1.values[i,j]) for j in range(15)])'

In [145]:
 """
Description     : Simple Python implementation of the Apriori Algorithm

Usage:
    $python apriori.py -f DATASET.csv -s minSupport  -c minConfidence

    $python apriori.py -f DATASET.csv -s 0.15 -c 0.6
"""

import sys

from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser


def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
    """calculates the support for items in the itemSet and returns a subset
    of the itemSet each of whose elements satisfies the minimum support"""
    _itemSet = set()
    localSet = defaultdict(int)

    for item in itemSet:
        for transaction in transactionList:
            if item.issubset(transaction):
                freqSet[item] += 1
                localSet[item] += 1

    for item, count in localSet.items():
        support = float(count) / len(transactionList)

        if support >= minSupport:
            _itemSet.add(item)

    return _itemSet


def joinSet(itemSet, length):
    """Join a set with itself and returns the n-element itemsets"""
    return set(
        [i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length]
    )


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))  # Generate 1-itemSets
    return itemSet, transactionList


def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet)

    currentLSet = oneCSet
    k = 2
    while currentLSet != set([]):
        largeSet[k - 1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(
            currentLSet, transactionList, minSupport, freqSet
        )
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
        """local function which Returns the support of an item"""
        return float(freqSet[item]) / len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item)) for item in value])

    toRetRules = []
    for key, value in list(largeSet.items())[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item) / getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)), confidence))
    return toRetItems, toRetRules


def printResults(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    for item, support in sorted(items, key=lambda x: x[1]):
        print("item: %s , %.3f" % (str(item), support))
    print("\n------------------------ RULES:")
    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        print("Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence))


def to_str_results(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    i, r = [], []
    for item, support in sorted(items, key=lambda x: x[1]):
        x = "item: %s , %.3f" % (str(item), support)
        i.append(x)

    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        x = "Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence)
        r.append(x)

    return i, r


def dataFromFile(fname):
    """Function which reads from the file and yields a generator"""
    with open(fname, "rU") as file_iter:
        for line in file_iter:
            line = line.strip().rstrip(",")  # Remove trailing comma
            record = frozenset(line.split(","))
            yield record


In [118]:
items, rules = runApriori('testr.csv',0.3,0.2)
printResults(items, rules)


------------------------ RULES:


In [89]:
import numpy as np
np.array(itemSetList1)

array([[39, ' State-gov', 77516, ..., 40, ' United-States', ' <=50K'],
       [50, ' Self-emp-not-inc', 83311, ..., 13, ' United-States',
        ' <=50K'],
       [38, ' Private', 215646, ..., 40, ' United-States', ' <=50K'],
       ...,
       [58, ' Private', 151910, ..., 40, ' United-States', ' <=50K'],
       [22, ' Private', 201490, ..., 20, ' United-States', ' <=50K'],
       [52, ' Self-emp-inc', 287927, ..., 40, ' United-States', ' >50K']],
      dtype=object)

In [53]:

#freqItemSet1, rules1 = apriori(np.array(itemSetList1), minSup=0.6, minConf=0.8)

In [167]:
import time
start=time.time()
itemSetList2=itemSetList1.iloc[:1000]
freqItemSet1, rules1 = apriori(np.array(itemSetList2), minSup=0.6, minConf=0.8)
end=time.time()
print("Time elapsed = ",end-start)

Time elapsed =  1.876664638519287


In [168]:
#freqItemSet1, rules1 = apriori(np.array(itemSetList2), minSup=0.6, minConf=0.8)

In [169]:
freqItemSet1

{1: {frozenset({' Private'}),
  frozenset({' White'}),
  frozenset({' <=50K'}),
  frozenset({' United-States'}),
  frozenset({' Male'}),
  frozenset({0})},
 2: {frozenset({' <=50K', ' United-States'}),
  frozenset({' <=50K', 0}),
  frozenset({' Private', ' United-States'}),
  frozenset({' Private', 0}),
  frozenset({' United-States', 0}),
  frozenset({' Male', ' United-States'}),
  frozenset({' Male', 0}),
  frozenset({' <=50K', ' White'}),
  frozenset({' United-States', ' White'}),
  frozenset({' White', 0})},
 3: {frozenset({' <=50K', ' United-States', 0}),
  frozenset({' Private', ' United-States', 0}),
  frozenset({' Male', ' United-States', 0}),
  frozenset({' <=50K', ' White', 0}),
  frozenset({' United-States', ' White', 0})}}

In [172]:
rules1

[]

In [48]:
pd.unique(itemSetList2[' 2174'])

TypeError: list indices must be integers or slices, not str

In [35]:
itemSetList2['39'].value_counts()

31    293
35    277
33    277
34    273
25    272
     ... 
84      3
82      2
83      2
88      1
85      1
Name: 39, Length: 71, dtype: int64

In [103]:
import time
start=time.time()
itemSetList2=itemSetList1
freqItemSet1, rules1 = apriori(np.array(itemSetList2), minSup=0.6, minConf=0.8)
end=time.time()
print("Time elapsed = ",end-start)

KeyboardInterrupt: 

In [100]:
freqItemSet1

{1: {frozenset({' Private'}),
  frozenset({' White'}),
  frozenset({' <=50K'}),
  frozenset({' United-States'}),
  frozenset({' Male'}),
  frozenset({0})},
 2: {frozenset({' <=50K', ' United-States'}),
  frozenset({' <=50K', 0}),
  frozenset({' Private', ' United-States'}),
  frozenset({' Private', 0}),
  frozenset({' United-States', 0}),
  frozenset({' Male', 0}),
  frozenset({' <=50K', ' White'}),
  frozenset({' United-States', ' White'}),
  frozenset({' White', 0})},
 3: {frozenset({' <=50K', ' United-States', 0}),
  frozenset({' United-States', ' White', 0}),
  frozenset({' <=50K', ' White', 0}),
  frozenset({' Private', ' United-States', 0})}}

In [101]:
rules1

[[{' <=50K'}, {' White'}, 0.8373381877022654],
 [{' <=50K'}, {' White', 0}, 0.8373381877022654],
 [{' <=50K', 0}, {' White'}, 0.8373381877022654],
 [{0}, {' White'}, 0.8542735173981143],
 [{' United-States'}, {' White'}, 0.8783339046966061],
 [{' United-States'}, {' White', 0}, 0.8783339046966061],
 [{' United-States', 0}, {' White'}, 0.8783339046966061],
 [{' Private'}, {' United-States'}, 0.8871607331688404],
 [{' Private'}, {' United-States', 0}, 0.8871607331688404],
 [{' Private', 0}, {' United-States'}, 0.8871607331688404],
 [{' <=50K'}, {' United-States'}, 0.8899271844660194],
 [{' <=50K'}, {' United-States', 0}, 0.8899271844660194],
 [{' <=50K', 0}, {' United-States'}, 0.8899271844660194],
 [{0}, {' United-States'}, 0.895857006848684],
 [{' White'}, {' United-States'}, 0.9210885821110153],
 [{' White'}, {' United-States', 0}, 0.9210885821110153],
 [{' White', 0}, {' United-States'}, 0.9210885821110153],
 [{' <=50K'}, {0}, 1.0],
 [{' Private'}, {0}, 1.0],
 [{' United-States'}, {0

In [57]:
def fpgrowthFromFile(fname, minSupRatio, minConf):
    itemSetList, frequency = getFromFile(fname)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)

    freqItems = []
    mineTree(headerTable, minSup, set(), freqItems)
    rules = associationRule(freqItems, itemSetList, minConf)
    return freqItems, rules

In [58]:
def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    fpTree = Node('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    return fpTree, headerTable

def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]

def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode


In [59]:
def mineTree(headerTable, minSup, preFix, freqItemList):
    # Sort the items with frequency and create a list
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])] 
    # Start with the lowest frequency
    for item in sortedItemList:  
        # Pattern growth is achieved by the concatenation of suffix pattern with frequent patterns generated from conditional FP-tree
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        # Find all prefix path, constrcut conditional pattern base
        conditionalPattBase, frequency = findPrefixPath(item, headerTable) 
        # Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = constructTree(conditionalPattBase, frequency, minSup) 
        if newHeaderTable != None:
            # Mining recursively on the tree
            mineTree(newHeaderTable, minSup,
                       newFreqSet, freqItemList)

def findPrefixPath(basePat, headerTable):
    # First node in linked list
    treeNode = headerTable[basePat][1] 
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf node all the way to root
        ascendFPtree(treeNode, prefixPath)  
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next node
        treeNode = treeNode.next  
    return condPats, frequency

def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)


In [60]:
from collections import defaultdict, OrderedDict
from csv import reader
from itertools import chain, combinations
import time

class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.display(ind+1)

def getFromFile(fname):
    itemSetList = []
    frequency = []
    
    with open(fname, 'r') as file:
        csv_reader = reader(file)
        for line in csv_reader:
            line = list(filter(None, line))
            itemSetList.append(line)
            frequency.append(1)

    return itemSetList, frequency

def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    fpTree = Node('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=False)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    return fpTree, headerTable

def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]

def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)

def findPrefixPath(basePat, headerTable):
    # First node in linked list
    treeNode = headerTable[basePat][1] 
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf node all the way to root
        ascendFPtree(treeNode, prefixPath)  
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next node
        treeNode = treeNode.next  
    return condPats, frequency

def mineTree(headerTable, minSup, preFix, freqItemList):
    # Sort the items with frequency and create a list
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])] 
    print('Finish sorting:', sortedItemList)
    # Start with the lowest frequency
    for item in sortedItemList:  
        print('Checking item:', item)
        print('in', sortedItemList)
        # Pattern growth is achieved by the concatenation of suffix pattern with frequent patterns generated from conditional FP-tree
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        print('Adding new frequent set: ', newFreqSet)
        freqItemList.append(newFreqSet)
        # Find all prefix path, constrcut conditional pattern base
        conditionalPattBase, frequency = findPrefixPath(item, headerTable) 
        # Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = constructTree(conditionalPattBase, frequency, minSup) 
        print(newHeaderTable)
        if newHeaderTable != None:
            print('Conditional tree for item:', item)
            conditionalTree.display()
            print()
            # print('current item', item)
            # print('conditional tree for: ', newFreqSet)
            # conditionalTree.display(1)

            # Mining recursively on the tree
            mineTree(newHeaderTable, minSup,
                       newFreqSet, freqItemList)
        else:
            print()

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count

def associationRule(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        for s in subsets:
            confidence = float(getSupport(itemSet, itemSetList) / getSupport(s, itemSetList))
            if(confidence > minConf):
                rules.append([set(s), set(itemSet.difference(s)),
                              confidence, getSupport(itemSet, itemSetList), getSupport(s, itemSetList)])
    return rules



In [66]:
if __name__ == "__main__":
    minSupRatio = 0.6
    minConf = 0.8
    fname = 'testr'
    itemSetList, frequency = getFromFile(fname + '.csv')
    minSup = len(itemSetList) * minSupRatio
    startTime = time.time()
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    # fpTree.display()
    if(fpTree == None):
        print('No frequent item set')
    else:
        # fpTree.display()
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        print(time.time() - startTime)
        # print('Frequent patterns:')
        # for x in freqItems:
        #     print(x)

        # rules = associationRule(freqItems, itemSetList, minConf)
        # print('\nRules:')
        # for rule in rules:
        #     print('{} ==> {}   {:.3f}   sup: {:.3f}  subsetsup: {:.3f}'.format(rule[0], rule[1], rule[2], rule[3] / len(itemSetList), rule[4] / len(itemSetList)))


Finish sorting: [' Male', ' Private', ' <=50K', ' White', ' United-States', ' 0']
Checking item:  Male
in [' Male', ' Private', ' <=50K', ' White', ' United-States', ' 0']
Adding new frequent set:  {' Male'}
None

Checking item:  Private
in [' Male', ' Private', ' <=50K', ' White', ' United-States', ' 0']
Adding new frequent set:  {' Private'}
None

Checking item:  <=50K
in [' Male', ' Private', ' <=50K', ' White', ' United-States', ' 0']
Adding new frequent set:  {' <=50K'}
None

Checking item:  White
in [' Male', ' Private', ' <=50K', ' White', ' United-States', ' 0']
Adding new frequent set:  {' White'}
{' <=50K': [20699, <__main__.Node object at 0x0000025193D4BBC8>]}
Conditional tree for item:  White
   Null   1
      <=50K   20699

Finish sorting: [' <=50K']
Checking item:  <=50K
in [' <=50K']
Adding new frequent set:  {' <=50K', ' White'}
None

Checking item:  United-States
in [' Male', ' Private', ' <=50K', ' White', ' United-States', ' 0']
Adding new frequent set:  {' United-St

In [67]:
print('Frequent patterns:')
for x in freqItems:
    print(x)

Frequent patterns:
{' Male'}
{' Private'}
{' <=50K'}
{' White'}
{' <=50K', ' White'}
{' United-States'}
{' Private', ' United-States'}
{' <=50K', ' United-States'}
{' White', ' United-States'}
{' 0'}
{' 0'}
{' 0', ' Male'}
{' Private', ' 0'}
{' Private', ' 0'}
{' Private', ' 0', ' Male'}
{' 0', ' <=50K'}
{' 0', ' <=50K'}
{' 0', ' Male', ' <=50K'}
{' Private', ' 0', ' <=50K'}
{' Private', ' 0', ' Male', ' <=50K'}
{' 0', ' White'}
{' 0', ' White'}
{' 0', ' White', ' Male'}
{' Private', ' 0', ' White'}
{' Private', ' 0', ' White', ' Male'}
{' 0', ' White', ' <=50K'}
{' 0', ' White', ' Male', ' <=50K'}
{' Private', ' 0', ' White', ' <=50K'}
{' 0', ' United-States'}
{' 0', ' United-States'}
{' 0', ' Male', ' United-States'}
{' Private', ' 0', ' United-States'}
{' Private', ' 0', ' Male', ' United-States'}
{' 0', ' United-States', ' <=50K'}
{' 0', ' United-States', ' <=50K'}
{' 0', ' Male', ' United-States', ' <=50K'}
{' Private', ' 0', ' United-States', ' <=50K'}
{' 0', ' White', ' United-S

In [69]:
len(freqItems)

45

In [70]:
rules = associationRule(freqItems, itemSetList, minConf)
print('\nRules:')
for rule in rules:
    print('{} ==> {}   {:.3f}   sup: {:.3f}  subsetsup: {:.3f}'.format(rule[0], rule[1], rule[2], rule[3] / len(itemSetList), rule[4] / len(itemSetList)))



Rules:
{' <=50K'} ==> {' White'}   0.837   sup: 0.636  subsetsup: 0.759
{' Private'} ==> {' United-States'}   0.887   sup: 0.618  subsetsup: 0.697
{' <=50K'} ==> {' United-States'}   0.890   sup: 0.676  subsetsup: 0.759
{' White'} ==> {' United-States'}   0.921   sup: 0.787  subsetsup: 0.854
{' United-States'} ==> {' White'}   0.878   sup: 0.787  subsetsup: 0.896
{' Male'} ==> {' 0'}   1.000   sup: 0.669  subsetsup: 0.669
{' Private'} ==> {' 0'}   1.000   sup: 0.697  subsetsup: 0.697
{' Private'} ==> {' 0'}   1.000   sup: 0.697  subsetsup: 0.697
{' Private', ' Male'} ==> {' 0'}   1.000   sup: 0.459  subsetsup: 0.459
{' <=50K'} ==> {' 0'}   1.000   sup: 0.759  subsetsup: 0.759
{' <=50K'} ==> {' 0'}   1.000   sup: 0.759  subsetsup: 0.759
{' <=50K', ' Male'} ==> {' 0'}   1.000   sup: 0.465  subsetsup: 0.465
{' Private', ' <=50K'} ==> {' 0'}   1.000   sup: 0.545  subsetsup: 0.545
{' Private', ' <=50K', ' Male'} ==> {' 0'}   1.000   sup: 0.329  subsetsup: 0.329
{' 0'} ==> {' White'}   0.85

In [71]:
len(rules)

105

In [80]:
#from mlxtend.frequent_patterns import fpgrowth


In [74]:
!pip install pyfpgrowth

In [75]:
import pyfpgrowth

In [76]:
patterns = pyfpgrowth. find_frequent_patterns(itemSetList, 19536)

In [77]:
rules = pyfpgrowth. generate_association_rules(patterns,0.8)

In [78]:
patterns

{(' Male',): 21790,
 (' 0', ' Male'): 40341,
 (' Private', ' United-States'): 20135,
 (' 0', ' Private', ' United-States'): 37795,
 (' 0', ' Private'): 42678,
 (' 0', ' 0', ' Private'): 19982,
 (' <=50K', ' White'): 20699,
 (' 0', ' <=50K', ' White'): 39860,
 (' <=50K', ' United-States'): 21999,
 (' 0', ' <=50K', ' United-States'): 42393,
 (' 0', ' <=50K'): 47659,
 (' 0', ' 0', ' <=50K'): 22939,
 (' United-States', ' White'): 25621,
 (' 0', ' United-States', ' White'): 47718,
 (' 0', ' White'): 51877,
 (' 0', ' 0', ' White'): 24061,
 (' United-States',): 29170,
 (' 0', ' United-States'): 54490,
 (' 0',): 60891,
 (' 0', ' 0'): 28330}

In [79]:
rules

{(' Male',): ((' 0',), 1.8513538320330427),
 (' United-States',): ((' 0',), 1.868015083990401),
 (' 0', ' Private'): ((' United-States',), 0.8855850789634003),
 (' Private', ' United-States'): ((' 0',), 1.8770797119443754),
 (' 0', ' <=50K'): ((' United-States',), 0.8895067038754485),
 (' <=50K', ' White'): ((' 0',), 1.9256968935697376),
 (' <=50K', ' United-States'): ((' 0',), 1.9270421382790126),
 (' 0', ' 0'): ((' White',), 0.8493116837274973),
 (' 0', ' United-States'): ((' White',), 0.8757203156542485),
 (' 0', ' White'): ((' United-States',), 0.9198295969312027),
 (' United-States', ' White'): ((' 0',), 1.862456578587877),
 (' 0',): ((' United-States',), 0.8948777323413969)}

In [82]:
!pip install mlxtend

In [124]:
#libraries
from itertools import combinations, chain
from  more_itertools import unique_everseen
#import sets
import time
import random
#import data and preprocess
data = open("testr.csv","r")

#sampling improvement for Apriori
samplingfactor = .6



#input data and each observation as a list within a data list structure
#add _i to data value indicating i'th variable
def clean(dta,samplingfactor):
    datalist = []
    for line in dta:
        linesep = line.split(", ")
        for attribute in linesep:
            newattribute = attribute  + "_" + str(linesep.index(attribute))
            linesep[linesep.index(attribute)] = newattribute
        datalist.append(linesep)

    random.shuffle(datalist) #randomizes data order
    datalist = [datalist[i] for i in range(0,int(round(samplingfactor*len(datalist))))] #return the first sampfactor
    return datalist



#Obtaining C1 - counts of all variable values
def gen_C1(dta):
    count_dict = {}
    for observation in dta:
        for val in observation:

            #if value in observation is not in dictionary, start its count at 1
            #if it is in the dictionary then increment its count by 1

            if val not in count_dict:
                count_dict[val]= 1
            elif val in count_dict:
                count_dict[val] += 1

    return count_dict


#Obtaining L1
def gen_L1(dict,min_supp,samplingfactor):
    L1 = []
    for var in dict:

        #if a value in the count dictionary passes the minimum support threshold
        #add it to list
        if float(dict[var])/float(nobservations) > min_supp*samplingfactor:
            L1.append(var)

    return L1


#print nobservations


#check if k-1 subset of k itemset is in L_k-1
def has_infeq_subsets(k,L,c):
    for i in list(combinations(c,k-1)):
            if i not in L:
                return False
            else:
                return True

#generate C2 from L1
def gen_C2(k,L):
    Ck = list(combinations(L,k)) #all combinations of 2 itemsets
    for c in Ck:
        if has_infeq_subsets(k,L,c):
            Ck.remove(c) #if it has infrequent subsets then remove candidate
        else:
            pass
    return Ck


def gen_Lk(Ck,min_supp,dta,samplingfactor):
    countdict = {}
    for c in Ck:
        countdict[c] = 0 #start every candidate count at 0

    for observation in dta:
        for c in Ck:
            if set(c).issubset(observation):
                countdict[c] += 1 #if candidate is subset of observation, increment count

    Lk = []
    for c in Ck:
        if float(countdict[c])/nobservations > min_supp*samplingfactor:
            Lk.append(c) #if minimum support threshold is passed append to list

    return Lk

#generate candidates for k > 2
#flatten tuples, get all possible combinations, and prune
def gen_Ck(k,L):
    flatten = [item for subtuple in L for item in subtuple] #flattens all candidates
    uniqueflatten = list(unique_everseen(flatten)) #gets out duplicate candidates
    Ck = list(combinations(uniqueflatten,k)) #creates list of all possible combinations of k length
    for c in Ck:
        if has_infeq_subsets(k,L,c):
            Ck.remove(c)
        else:
            pass
    return Ck


#Apriori Algorithm method
def Apriori(k,L,dta,min_supp,samplingfactor):
    l_of_L = [] #to store different Lk's
    while L != []: #while there are frequent itemsets
        if k == 2:
            Ck = gen_C2(k,L) #use specific 2-tuple candidate generator
        else:
            Ck = gen_Ck(k,L) #otherwise use normal one
        L = gen_Lk(Ck,min_supp,cleandata,samplingfactor)
        l_of_L.append(L)
        k += 1

    return l_of_L[0:len(l_of_L)-1] #return all Lk stored that are non-empty


###Test###

time1 = time.time()

cleandata = clean(data,samplingfactor)

#number of observations, used later for min_supp testing
nobservations = len(cleandata)

#getfirst candidate set
C1 = gen_C1(cleandata)

#generate L1
L1 = gen_L1(C1,.75,samplingfactor)
freqsets = Apriori(2,L1,cleandata,.6,samplingfactor)

print("The frequent itemsets are:" + "\n")
for i in freqsets:
    for j in i:
        print(j)

time2 = time.time()

testtime = time2 - time1
print("\n")
print("The runtime for this algorithm(with a sampling factor of .6 and a min_supp = .6) is" + " " + str(testtime) + " seconds.")


The frequent itemsets are:

('Married-civ-spouse_5', 'White_8')
('Married-civ-spouse_5', 'Male_9')
('Married-civ-spouse_5', '0_10')
('Married-civ-spouse_5', '0_11')
('Married-civ-spouse_5', 'United-States_13')
('White_8', 'Male_9')
('White_8', '0_10')
('White_8', '0_11')
('White_8', 'United-States_13')
('White_8', '<=50K\n_14')
('White_8', 'Private_1')
('White_8', '40_12')
('Male_9', '0_10')
('Male_9', '0_11')
('Male_9', 'United-States_13')
('Male_9', '<=50K\n_14')
('Male_9', 'Private_1')
('0_10', '0_11')
('0_10', 'United-States_13')
('0_10', '<=50K\n_14')
('0_10', 'Private_1')
('0_10', '40_12')
('0_11', 'United-States_13')
('0_11', '<=50K\n_14')
('0_11', 'Private_1')
('0_11', '40_12')
('United-States_13', '<=50K\n_14')
('United-States_13', 'Private_1')
('United-States_13', '40_12')
('<=50K\n_14', 'Private_1')
('<=50K\n_14', '40_12')
('Married-civ-spouse_5', 'White_8', '0_10')
('Married-civ-spouse_5', 'White_8', 'United-States_13')
('Married-civ-spouse_5', 'Male_9', '0_10')
('Married-c

## BEst one


In [129]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 12 10:50:51 2016
@author: nancynan
"""

from collections import defaultdict

import time
#os.system('clear')

class Apriori:
    
    
    def __init__(self, input_file, min_sup):
        self.input_file = input_file
        self.min_sup = min_sup
        self.translist = []
        self.items_with_sup = []
        self.freqset = defaultdict(int)
        self.enlarged = {}
    
    def _readFile(self):
        with open(self.input_file, 'rU') as f:
            for line in f:
                #print(line)
                lines = line.strip().split(',')
                lines[0] = 'age:'+lines[0]
                lines[1] = 'workclass:'+lines[1]
                lines[2] = 'fnlwgt:'+lines[2]
                lines[3] = 'education:'+lines[3]
                lines[4] = 'ed_num:'+lines[4]
                lines[5] = 'marital-status:'+lines[5]
                lines[6] = 'occupation:'+lines[6]
                lines[7] = 'relationship:'+lines[7]
                lines[8] = 'race:'+lines[8]
                lines[9] = 'sex:'+lines[9]
                lines[10] = 'capital-gain:'+lines[10]
                lines[11] = 'capital-loss:'+lines[11]
                lines[12] = 'hrs-per-week:'+lines[12]
                lines[13] = 'native-country:'+lines[13]
                lines[14] = 'salary:'+lines[14]
                yield lines
                
    def find_C1(self):
        C1=set()
        for row in self._readFile():
            rowi=frozenset(row)
            self.translist.append(rowi)
            for i in rowi:
                C1.add(frozenset([i]))
        return C1
                    
    
    def prunestep(self,C):
        L=set()
        freqsetloc = defaultdict(int)

        for item in C:
            for trans in self.translist:
                if item.issubset(trans):
                    self.freqset[item]+=1
                    freqsetloc[item]+=1
        for key,value in freqsetloc.items():
            support=float(value)/len(self.translist)
            if support>=min_sup:
                L.add(key)
        return L
        
    def Support(self, item):
        return float(self.freqset[item]) / len(self.translist)    
        
    def joinstep(self, itemset, length):
        Ck=set()
        for i in itemset:
            for j in itemset:
                joinres=i.union(j)
                if len(joinres)==length:
                    Ck.add(joinres)
        return Ck
                
    def run(self):
        C1=self.find_C1()
        Lk=self.prunestep(C1)
        k=2
        empset=set([])

        while (Lk!=empset):
            self.enlarged[k-1]=Lk
            nextC=self.joinstep(Lk,k)
            nextL=self.prunestep(nextC)
            Lk=nextL
            k+=1
            
        for value in self.enlarged.values():
            for item in value:
                self.items_with_sup.append(
                (tuple(item), self.Support(item)))
                    
    def printitembysup(self):
            i = 1
            for item, support in sorted(
            self.items_with_sup, key=lambda i: i[1],
            reverse=True):
                print ('Item %d: %s, Support: %.3f' % (i, str(item), support))
                i += 1
            
    def writefreqpat(self):
        with open('apriori_output.csv', 'w') as f:
            for item, support in sorted(
            self.items_with_sup, key=lambda i: i[1],
            reverse=True):
                f.write('%.3f, %s\n' % (support, ','.join(item)))
    

if __name__ == '__main__':
#    begin1=time.time()
    input_file = 'testr.csv'
    
    #!!you can change into any number in (0,1)
    min_sup = 0.5

    a = Apriori(input_file, min_sup)
    a.run()
    a.printitembysup()

    # !!add # below if don't want csv file
    a.writefreqpat()
#    stop1=time.time()
#    print ('apriori takes',(stop1-begin1))


C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: 'U' mode is deprecated


IndexError: list index out of range

In [130]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 19 10:50:51 2016

@author: nancynan
"""
#import sys
from collections import defaultdict
#import os
import time
#os.system('clear')

class Apriori:
    
    
    def __init__(self, input_file, min_sup):
        self.input_file = input_file
        self.min_sup = min_sup
        self.translist = []
        self.items_with_sup = []
        self.freqset = defaultdict(int)
        self.enlarged = {}
        self.L1_count = {} 
        self.L1_TID = {}
    
    def _readFile(self):
        with open(self.input_file, 'rU') as f:
            for line in f:
#                print(line)
                lines = line.strip().split(',')
                lines[0] = 'age:'+lines[0]
                lines[1] = 'workclass:'+lines[1]
                lines[2] = 'fnlwgt:'+lines[2]
                lines[3] = 'education:'+lines[3]
                lines[4] = 'ed_num:'+lines[4]
                lines[5] = 'marital-status:'+lines[5]
                lines[6] = 'occupation:'+lines[6]
                lines[7] = 'relationship:'+lines[7]
                lines[8] = 'race:'+lines[8]
                lines[9] = 'sex:'+lines[9]
                lines[10] = 'capital-gain:'+lines[10]
                lines[11] = 'capital-loss:'+lines[11]
                lines[12] = 'hrs-per-week:'+lines[12]
                lines[13] = 'native-country:'+lines[13]
                yield lines
                
    def find_C1(self):
        C1=set()
        tid=0
        for row in self._readFile():
            rowi=frozenset(row)
            self.translist.append(rowi)
            tid += 1
            for i in rowi:
                C1.add(frozenset([i]))
                if i not in self.L1_TID:
                    self.L1_TID[i]=set([tid])
                else:
                    self.L1_TID[i].add(tid)
        return C1
                    
    
    def prunestep(self,C):
        L=set()
        freqsetloc = defaultdict(int)

        for item in C:
            if all(x in self.L1_count for x in list(item)):
                min_item = self.findMinItem(item)
                # TID:list of transaaction IDs for min_item
                TID = self.L1_TID[min_item]
                # only scan those with id in TID
                for index in TID:
                    if item.issubset(self.translist[index-1]):
                        self.freqset[item] += 1
                        freqsetloc[item] += 1
        for key,value in freqsetloc.items():
            support=float(value)/len(self.translist)
            if support>=min_sup:
                L.add(key)
        return L
        
    def findMinItem(self, items):
        # split into single item and return them with min_sup
        minimum =100000
        for key,value in self.L1_count.items():
            if key in items and value < minimum:
                minimum = value
        for key,value in self.L1_count.items():
            if value == minimum:
                return key 

    def getL1Count(self, C1):
        # for content of dictionary L1_count
        local_set = defaultdict(int)
        for item in C1:
            for rowi in self.translist:
                if item.issubset(rowi):
                    local_set[item] += 1
        for item, count in local_set.items():
            support = float(count)/len(self.translist)
            if support >= min_sup:
                key = [k for k in item][0]
                self.L1_count[key]=count
        return 
        
    def Support(self, item):
        return float(self.freqset[item]) / len(self.translist)    
        
    def joinstep(self, itemset, length):
        Ck=set()
        for i in itemset:
            for j in itemset:
                joinres=i.union(j)
                if len(joinres)==length:
                    Ck.add(joinres)
        return Ck
                
    def run(self):
        C1=self.find_C1()
        self.getL1Count(C1)
        Lk=self.prunestep(C1)
        k=2
        empset=set([])

        while (Lk!=empset):
            self.enlarged[k-1]=Lk
            nextC=self.joinstep(Lk,k)
            nextL=self.prunestep(nextC)
            Lk=nextL
            k+=1
            
        for value in self.enlarged.values():
            for item in value:
                self.items_with_sup.append(
                (tuple(item), self.Support(item)))
                    
    def printitembysup(self):
            i = 1
            for item, support in sorted(
            self.items_with_sup, key=lambda i: i[1],
            reverse=True):
                print ('Item %d: %s, Support: %.3f' % (i, str(item), support))
                i += 1
            
    def writefreqpat(self):
        with open('improved_apriori_output.csv', 'w') as f:
            for item, support in sorted(
                self.items_with_sup, key=lambda i: i[1],
                reverse=True):
                f.write('%.3f, %s\n' % (support, ','.join(item)))
    

if __name__ == '__main__':
    begin1=time.time()
    input_file = 'adult_data.csv'
    
    #!!you can change into any number in (0,1)
    min_sup = 0.5

    a = Apriori(input_file, min_sup)
    a.run()
    a.printitembysup()

    # !!add # below if don't want csv file
    a.writefreqpat()
    stop1=time.time()
    print ('imroved_fp takes',(stop1-begin1))


C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: 'U' mode is deprecated


Item 1: ('capital-loss: 0',), Support: 0.953
Item 2: ('capital-gain: 0',), Support: 0.917
Item 3: ('native-country: United-States',), Support: 0.896
Item 4: ('capital-gain: 0', 'capital-loss: 0'), Support: 0.870
Item 5: ('race: White',), Support: 0.854
Item 6: ('native-country: United-States', 'capital-loss: 0'), Support: 0.854
Item 7: ('native-country: United-States', 'capital-gain: 0'), Support: 0.820
Item 8: ('race: White', 'capital-loss: 0'), Support: 0.813
Item 9: ('race: White', 'native-country: United-States'), Support: 0.787
Item 10: ('race: White', 'capital-gain: 0'), Support: 0.780
Item 11: ('native-country: United-States', 'capital-gain: 0', 'capital-loss: 0'), Support: 0.778
Item 12: (' <=50K',), Support: 0.759
Item 13: ('race: White', 'native-country: United-States', 'capital-loss: 0'), Support: 0.748
Item 14: ('race: White', 'capital-gain: 0', 'capital-loss: 0'), Support: 0.739
Item 15: (' <=50K', 'capital-loss: 0'), Support: 0.736
Item 16: (' <=50K', 'capital-gain: 0'), 

In [173]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 3, 03:09:11 2021

@author: ayushi
"""

"""
APRIORI ALGORITHM
"""

import sys

from itertools import chain, combinations
from collections import defaultdict
import pandas as pd

def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
    """calculates the support for items in the itemSet and returns a subset
    of the itemSet each of whose elements satisfies the minimum support"""
    _itemSet = set()
    localSet = defaultdict(int)

    for item in itemSet:
        for transaction in transactionList:
            if item.issubset(transaction):
                freqSet[item] += 1
                localSet[item] += 1

    for item, count in localSet.items():
        support = float(count) / len(transactionList)

        if support >= minSupport:
            _itemSet.add(item)

    return _itemSet


def joinSet(itemSet, length):
    """Join a set with itself and returns the n-element itemsets"""
    return set(
        [i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length]
    )


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))  # Generate 1-itemSets
    return itemSet, transactionList


def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet)

    currentLSet = oneCSet
    k = 2
    while currentLSet != set([]):
        largeSet[k - 1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(
            currentLSet, transactionList, minSupport, freqSet
        )
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
        """local function which Returns the support of an item"""
        return float(freqSet[item]) / len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item)) for item in value])

    toRetRules = []
    for key, value in list(largeSet.items())[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item) / getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)), confidence))
    return toRetItems, toRetRules


def printResults(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    for item, support in sorted(items, key=lambda x: x[1]):
        print("item: %s , %.3f" % (str(item), support))
    print("\n------------------------ RULES:")
    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        print("Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence))


def to_str_results(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    i, r = [], []
    for item, support in sorted(items, key=lambda x: x[1]):
        x = "item: %s , %.3f" % (str(item), support)
        i.append(x)

    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        x = "Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence)
        r.append(x)

    return i, r


def dataFromFile(fname):
    """Function which reads from the file and yields a generator"""
    with open(fname, "rU") as file_iter:
        for line in file_iter:
            line = line.strip().rstrip(",")  # Remove trailing comma
            record = frozenset(line.split(","))
            yield record


if __name__ == "__main__":
    
     inFile = dataFromFile('adult_data.csv')
     minSupport = 0.6
     minConfidence = 0.6
     
     items, rules = runApriori(inFile, minSupport, minConfidence)
     printResults(items, rules)


C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:140: DeprecationWarning: 'U' mode is deprecated


KeyboardInterrupt: 